In [1]:
import spotipy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import re
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import unidecode
import string
from fuzzywuzzy import fuzz

tqdm_notebook.pandas()

load_dotenv('../secrets/.env', override=True)

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

%run useful_functions.ipynb
%run spotify_scraping.ipynb

/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
broadcast_playlists = get_broadcast_playlists()
search = get_spotify_search()
tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()

tracks_to_run = broadcast_playlists[['Track Name', 'Artist Name']].sort_values(['Artist Name','Track Name'], ascending =  True).drop_duplicates(subset=['Artist Name','Track Name'], keep='first')
try:   
    tracks_to_run = tracks_to_run.merge(search[['Original Search Artist','Original Search Track']], how='left', left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], copy=False,indicator=True)
    tracks_to_run = tracks_to_run[tracks_to_run['_merge']!="both"][['Artist Name','Track Name']]
except:
    tracks_to_run = tracks_to_run[['Artist Name','Track Name']]
tracks_to_run

,Artist Name,Track Name
6,,Hamilton Bohannon - Disco Stomp
10,,Loftopus performer: GOLDEN AGE OF STEAM
11,,Nacholbedurfnis
12,,Off The Beach performer: WIRE label: Pink Flag...
14,,Public Enemy - Bring The Noise
...,...,...
56583,宮下富実夫,See The Light (Abridged)
56584,暗黒神話,cowbel song (Lomax Remix)
56585,爱乐团,Only She Knows
56587,跡部進一,Heat 1


In [3]:
max_iter = 1000

for i, (index, row) in enumerate(tracks_to_run[100:max_iter].iterrows()):
    print("Iteration ",i+1," of ",len(tracks_to_run.index),":",row['Artist Name'],":", row['Track Name'])
    try:
        (temp_tracks,temp_albums,temp_artists,temp_search) = get_track_info(row['Artist Name'],row['Track Name'])
    except:
        (temp_tracks,temp_albums,temp_artists,temp_search) = (pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame())
    tracks = pd.concat([tracks,temp_tracks])
    albums = pd.concat([albums,temp_albums])
    artists = pd.concat([artists,temp_artists])
    search = pd.concat([search,temp_search])
    
del(i,
    max_iter,
    index,
    row,
    temp_albums,
    temp_artists,
    temp_tracks,
    temp_search,
    tracks_to_run)  

Iteration  1  of  22450 : Agaskodo Teliverek : 30000 Lightyears Old


In [42]:
search = search.drop_duplicates(subset=['Original Search Artist', 'Original Search Track'], keep='last')
albums = albums.drop_duplicates(subset='Album ID', keep='last')
tracks = tracks.drop_duplicates(subset='Track ID', keep='last')
artists = artists.drop_duplicates(subset='Artist ID', keep='last')

In [43]:
search.drop(columns = ['Accept'],errors='ignore', inplace=True)
temp_search = search.merge(tracks, how = 'left', on = 'Track ID')[['Track ID', 'Track Name','Original Search Track', 'Cleaned Search Track', 'Original Search Artist']]
temp_search[['Track Name Lower', 'Original Search Track Lower']] = temp_search[['Track Name', 'Original Search Track']].applymap(lambda s: s.lower() if type(s) == str else s)
temp_search['Partial Ratio'] = temp_search.apply(lambda x: fuzz.partial_ratio(x['Track Name Lower'], x['Original Search Track Lower']), axis=1)
temp_search['Ratio'] = temp_search.apply(lambda x: fuzz.ratio(x['Track Name Lower'], x['Original Search Track Lower'],), axis=1)
temp_search['WRatio'] = temp_search.apply(lambda x: fuzz.WRatio(x['Track Name Lower'], x['Original Search Track Lower']), axis=1)
temp_search['Accept'] = temp_search.apply(lambda x: x['WRatio']>50 and x['Ratio'] > 50 and x['Partial Ratio'] > 50, axis=1)
search = search.merge(temp_search[['Original Search Artist','Original Search Track','Track ID', 'Accept']], how = 'left', on = ['Original Search Artist','Original Search Track','Track ID'])
  
temp_search.to_excel('../data/temp.xlsx')

In [44]:
tracks.to_pickle("../data/spotify_tracks.pkl")
albums.to_pickle("../data/spotify_albums.pkl")
artists.to_pickle("../data/spotify_artists.pkl")
search.to_pickle("../data/spotify_search.pkl")